In [1]:
# -------------------------
# 1) Mount Google Drive
# -------------------------
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# =========================
# COLAB: TIFF (8-bit) -> PNG
# 3 INPUT FOLDERS -> 3 OUTPUT FOLDERS (SAME NAMES)
# =========================
# What it does:
# - Reads all 8-bit TIFF images from 3 input folders on Google Drive
# - Saves them as PNG (no rescaling, no modification)
# - Preserves folder structure (same folder names)
# - Creates output folders automatically if missing

!pip -q install opencv-python-headless tqdm

import os
from pathlib import Path
import cv2
from tqdm import tqdm

# -------------------------
# 1) Mount Google Drive
# -------------------------
from google.colab import drive
drive.mount('/content/drive')

# -------------------------
# 2) CONFIG
# -------------------------
INPUT_FOLDERS = [
    "/content/drive/MyDrive/soil_microCT_images/ROI/bnei_reem_ROI_8bit",
    "/content/drive/MyDrive/soil_microCT_images/ROI/mishmar_hanegev_maoz_ROI_8bit",
    "/content/drive/MyDrive/soil_microCT_images/ROI/rehovot_ROI_8bit",
]

OUTPUT_ROOT = "/content/drive/MyDrive/soil_microCT_images/ROI/png_for_roboflow"

# Accepted TIFF extensions
TIFF_EXTS = {".tif", ".tiff"}

# -------------------------
# Helpers
# -------------------------
def ensure_dir(path: str):
    Path(path).mkdir(parents=True, exist_ok=True)

def list_tiffs(folder: str):
    folder = Path(folder)
    if not folder.exists():
        return []
    return sorted([
        str(f) for f in folder.iterdir()
        if f.is_file() and f.suffix.lower() in TIFF_EXTS
    ])

# -------------------------
# 3) Main conversion
# -------------------------
ensure_dir(OUTPUT_ROOT)

for in_dir in INPUT_FOLDERS:
    in_dir = in_dir.strip()
    if not in_dir:
        continue

    folder_name = Path(in_dir).name
    out_dir = os.path.join(OUTPUT_ROOT, folder_name)

    ensure_dir(out_dir)

    tiffs = list_tiffs(in_dir)

    print(f"\n=== INPUT FOLDER ===")
    print(in_dir)
    print(f"Found {len(tiffs)} TIFF files")
    print(f"Output -> {out_dir}")

    for tif_path in tqdm(tiffs, desc=f"Converting {folder_name}"):
        # Read image exactly as stored (no normalization)
        img = cv2.imread(tif_path, cv2.IMREAD_UNCHANGED)

        if img is None:
            print(f"WARNING: cannot read {tif_path}")
            continue

        # Safety check: ensure 8-bit
        if img.dtype != "uint8":
            print(f"WARNING: {tif_path} is not uint8 (dtype={img.dtype}), skipped")
            continue

        # Save as PNG
        out_name = Path(tif_path).stem + ".png"
        out_path = os.path.join(out_dir, out_name)
        cv2.imwrite(out_path, img)

print("\nDone ✅ All TIFF files converted to PNG")
print("Output root:", OUTPUT_ROOT)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

=== INPUT FOLDER ===
/content/drive/MyDrive/soil_microCT_images/ROI/bnei_reem_ROI_8bit
Found 652 TIFF files
Output -> /content/drive/MyDrive/soil_microCT_images/ROI/png_for_roboflow/bnei_reem_ROI_8bit


Converting bnei_reem_ROI_8bit: 100%|██████████| 652/652 [00:32<00:00, 20.16it/s]



=== INPUT FOLDER ===
/content/drive/MyDrive/soil_microCT_images/ROI/mishmar_hanegev_maoz_ROI_8bit
Found 1203 TIFF files
Output -> /content/drive/MyDrive/soil_microCT_images/ROI/png_for_roboflow/mishmar_hanegev_maoz_ROI_8bit


Converting mishmar_hanegev_maoz_ROI_8bit: 100%|██████████| 1203/1203 [01:08<00:00, 17.50it/s]



=== INPUT FOLDER ===
/content/drive/MyDrive/soil_microCT_images/ROI/rehovot_ROI_8bit
Found 661 TIFF files
Output -> /content/drive/MyDrive/soil_microCT_images/ROI/png_for_roboflow/rehovot_ROI_8bit


Converting rehovot_ROI_8bit: 100%|██████████| 661/661 [00:35<00:00, 18.48it/s]


Done ✅ All TIFF files converted to PNG
Output root: /content/drive/MyDrive/soil_microCT_images/ROI/png_for_roboflow
